# Initial data cleaning

This file creates a condensed metadata file from all the bills in the 2023-2024_118th_Congress folder.

In [1]:
# Modules

# To list files in a directory
import os

# Time for execution of the code
import time # for debugging

# For .json files
import json

# For data frames
import pandas as pd

# Count dictionary values
from collections import Counter

# Pick a random item from a list
import random

# Print dictionaries prettily
import pprint

In [2]:
# Functions

# Takes in a politician's id (as an integer) and makes a dictionary with the politicians's
# name, party, and state.
def politician_info(politician_id):
    with open('2023-2024_118th_Congress\\people\\'+str(politician_id)+'.json', 'r') as voter_file:
        voter_obj = json.load(voter_file)
        info = {
            "name":voter_obj["person"]["name"],
            "party":voter_obj["person"]["party"],
            "state":voter_obj["person"]["district"][3:5],
        }
        return info

# Takes in a vote's id (as an integer) and gets the votes, with parties and states, and the counts, 
# as a dictionary.
def roll_call_info(roll_call_id):
    with open('2023-2024_118th_Congress\\vote\\'+str(roll_call_id)+'.json', 'r') as vote_file:
        vote_obj = json.load(vote_file)
        info = {
            "people":[]
        }
        # Fill in the people dictionaries
        roll_call_votes = vote_obj["roll_call"]["votes"]
        for vote in roll_call_votes:   
            rc_vt_index = roll_call_votes.index(vote)
            voter = politician_info(roll_call_votes[rc_vt_index]["people_id"])
            voter["vote"] = roll_call_votes[rc_vt_index]["vote_text"]
            info["people"].append(voter)
        # Add the numbers of each type of vote for each party
        c_yea = Counter()
        c_nay = Counter()
        c_absent = Counter()
        c_nv = Counter()
        for item in info["people"]:
            if item["vote"] == "Yea":
                c_yea[item["party"]] += 1
            if item["vote"] == "Nay":
                c_nay[item["party"]] += 1
            if item["vote"] == "Absent":
                c_absent[item["party"]] += 1   
            if item["vote"] == "NV":
                c_nv[item["party"]] += 1    
        info["Yea"] = dict(c_yea)
        info["Nay"] = dict(c_nay)
        info["Absent"] = dict(c_absent)
        info["NV"] = dict(c_nv)
        return info

Given a bill, the following function creates a dictionary with the bill's: 
- chamber
- bill type 
- title
- summary
- introduction date
- sponsors by name, party, and state
- subject(s)
- number of actions in Congress
- latest action
- passage in House with date and votes
- passage in Senate with date and votes
- whether or not it was signed into law and the date

In [3]:
# Functions, cont.

# Create a dictionary item for a bill, given its name in the file format given by legiscan, no extension.
def bill_metadata(bill_name):   
    with open("2023-2024_118th_Congress\\bill\\"+bill_name+".json", 'r') as bill_file:
        # Convert to a python object
        obj = json.load(bill_file)

        # Initialize a dictionary with the data I want about the bill
        bill = {
            "chamber":obj["bill"]["body"], 
            "bill_type":obj["bill"]["bill_type"],
            "title":obj["bill"]["bill_number"]+":  "+obj["bill"]["title"],
            "summary":obj["bill"]["description"],
            "introduced":obj["bill"]["history"][0]["date"]
        }

        # Sponsor(s)
        # Get the people with their party and state
        bill["sponsor(s)"]={
            "people":[]
        }
        sps = obj["bill"]["sponsors"]
        for sponsor in sps:
            sp_index = sps.index(sponsor)
            with open('2023-2024_118th_Congress\\people\\'+str(sps[sp_index]["people_id"])+'.json', 'r') as voter_file:
                    voter_obj = json.load(voter_file)

                    # Append dictionary
                    bill["sponsor(s)"]["people"].append(politician_info(sps[sp_index]["people_id"]))   
        # Add the number of each party
        c_party = Counter()
        for item in bill["sponsor(s)"]["people"]:
            c_party[item["party"]] += 1
        bill["sponsor(s)"].update(c_party)            

        # Add the subject(s) of the bill
        bill["subject(s)"]=[]
        for subject in obj["bill"]["subjects"]:
            bill["subject(s)"].append(subject["subject_name"])

        # Number of actions and latest action
        bill["actions"] = len(obj["bill"]["history"])
        bill["latest_action"] = obj["bill"]["history"][len(obj["bill"]["history"])-1]["action"]

        # Phase
        
        hist = obj["bill"]["history"]
        bill["pass_house"] = {}
        bill["pass_senate"] = {}
        bill["became_law"]= {}
        
        # Find in the history whether it passed
        house_vote = list(filter(lambda event: (event["action"].startswith("On passage Passed") or
            "Agreed to by the Yeas and Nays" in event["action"]
            or "Agreed to by voice vote" in event["action"]) and event["chamber"]== "H", hist))
        senate_vote = list(filter(lambda event: event["action"].startswith("Passed Senate") 
            or "passed without amendment by Unanimous Consent" in event["action"]
            or ("passed" in event["action"] and event["chamber"] == "S"), hist))
        signed_by_pres = list(filter(lambda event: event["action"].startswith("Became Public Law"), hist))
        
        # Date, if passed
        # House
        bill["pass_house"]["passed"] = (len(house_vote) != 0)
        if bill["pass_house"]["passed"] == True:
            bill["pass_house"]["date"] = house_vote[0]["date"]
        else:
            bill["pass_house"]["date"] = ""
        # Senate
        bill["pass_senate"]["passed"] = (len(senate_vote) != 0)
        if bill["pass_senate"]["passed"] == True:
            bill["pass_senate"]["date"] = senate_vote[0]["date"]
        else:
            bill["pass_senate"]["date"] = ""
        # Law
        bill["became_law"]["signed"] = (len(signed_by_pres) != 0)
        if bill["became_law"]["signed"] == True:
            bill["became_law"]["date"] = signed_by_pres[0]["date"]
        else:
            bill["became_law"]["date"] = ""
        
        # Votes, if passed   
        votes = obj["bill"]["votes"]
        # House
        vote_to_pass_house = list(filter(lambda roll_call: 
            (roll_call["desc"].startswith("On Passage") 
            or roll_call["desc"].startswith("On Motion to Suspend the Rules and Pass"))
            and roll_call["chamber"]=="H", votes))
        if len(vote_to_pass_house) != 0:
            #print("vote_to_pass_house:") # for debugging
            #print(vote_to_pass_house) # for debugging
            vt_index = votes.index(vote_to_pass_house[0])
            bill["pass_house"]["votes"] = roll_call_info(votes[vt_index]["roll_call_id"])
        elif bill["pass_house"]["passed"] == True:
            bill["pass_house"]["votes"] = "Voice vote"
        else:
            bill["pass_house"]["votes"] = {}
        # Senate
        vote_to_pass_senate = list(filter(lambda roll_call: 
            (roll_call["desc"].startswith("On Passage") or roll_call["desc"].startswith("On the Joint Resolution")) 
            and roll_call["chamber"]=="S", votes))
        if len(vote_to_pass_senate) != 0:
            #print("vote_to_pass_senate:") # for debugging
            #print(vote_to_pass_senate) # for debugging
            vt_index = votes.index(vote_to_pass_senate[0])
            bill["pass_senate"]["votes"] = roll_call_info(votes[vt_index]["roll_call_id"])
        else:
            if bill["pass_senate"]["passed"] == True:
                bill["pass_senate"]["votes"] = {
                    "people":"Unanimous",
                    "Yea":{"D":48, "R":49, "I":3},
                    "Nay":{},
                    "Absent":{},
                    "NV":{}
                }
            else:
                bill["pass_senate"]["votes"] = {}
            
        # Return
        return bill

We can list all the bills, or at least count them.

In [4]:
# List all the bills

raw_bills_files_list = os.listdir("2023-2024_118th_Congress\\bill")
#print(raw_bills_files_list) # for debugging
print("There are "+str(len(raw_bills_files_list))+" bills.") # for debugging

There are 15366 bills.


We can test the bill_metadata function by inputting a random bill or a bill of our choice.

In [5]:
# Choose a bill
which_bill = "HB3935" #input("Choose a bill (input the appreviation according to the file name format, \
#without the extension) or type Random for a random bill.\n") 
if which_bill == "Random":
    selected_bill = random.choice(raw_bills_files_list).split(".")[0]
else:    
    selected_bill = which_bill 
selected_index = raw_bills_files_list.index(selected_bill+".json")

# Print the original file
print("Selected the "+str(selected_index+1)+"th bill, "+selected_bill+".\n")
print("Here is the original file:")
with open("2023-2024_118th_Congress\\bill\\"+selected_bill+".json", 'r') as bill_file:
    # Convert to a python object
    obj = json.load(bill_file)

    # See the contents of bill_file, formatted nicely
    json_formatted_str = json.dumps(obj, indent=4) 
    print(json_formatted_str) 
    
# Print my metadata
print("\nHere is my metadata:")
pprint.pprint(bill_metadata(selected_bill), sort_dicts=False)

Selected the 3258th bill, HB3935.

Here is the original file:
{
    "bill": {
        "bill_id": 1771174,
        "change_hash": "59c65a12539af3dead17490d98cdbc5b",
        "session_id": 2041,
        "session": {
            "session_id": 2041,
            "state_id": 52,
            "year_start": 2023,
            "year_end": 2024,
            "prefile": 0,
            "sine_die": 0,
            "prior": 0,
            "special": 0,
            "session_tag": "Regular Session",
            "session_title": "2023-2024 Regular Session",
            "session_name": "118th Congress"
        },
        "url": "https://legiscan.com/US/bill/HB3935/2023",
        "state_link": "https://www.congress.gov/bill/118th-congress/house-bill/3935/all-info",
        "completed": 0,
        "status": 2,
        "status_date": "2023-07-25",
        "progress": [
            {
                "date": "2023-06-09",
                "event": 1
            },
            {
                "date": "2023-06-09

                                      'vote': 'Yea'},
                                     {'name': 'Mary Gay Scanlon',
                                      'party': 'D',
                                      'state': 'PA',
                                      'vote': 'Absent'},
                                     {'name': 'Susan Wild',
                                      'party': 'D',
                                      'state': 'PA',
                                      'vote': 'Yea'},
                                     {'name': 'Colin Allred',
                                      'party': 'D',
                                      'state': 'TX',
                                      'vote': 'Yea'},
                                     {'name': 'Kelly Armstrong',
                                      'party': 'R',
                                      'state': 'ND',
                                      'vote': 'Yea'},
                                     {'name': 'James B

The following loop creates a list of dictionaries containing all the bills' metadata.

Note, the function didn't catch that HB3935 and SB546 have become law, since they haven't been published yet.  HB3935 became law on 16 May 2024 and SB546 became law on 24 May 2024.  We can manually add that afterward.

In [6]:
bills = []
total_time = 0
for file in raw_bills_files_list:
    start = time.time() # for debugging
    print("Bill "+file.split(".")[0]+":") # for debugging
    # Use the bill metadata function
    bill = bill_metadata(file.split(".")[0])
    # Append the bill and time elapsed.
    bills.append(bill)
    end = time.time() # for debugging
    total_time = total_time + end-start
    #print("Time elapsed for this bill:", end-start, "seconds\n") # for debugging
    print("Total time elapsed:", total_time, "seconds\n")

Bill HB1:
Total time elapsed: 0.037610769271850586 seconds

Bill HB100:
Total time elapsed: 0.03953218460083008 seconds

Bill HB1000:
Total time elapsed: 0.04053020477294922 seconds

Bill HB1001:
Total time elapsed: 0.04153108596801758 seconds

Bill HB1002:
Total time elapsed: 0.046619415283203125 seconds

Bill HB1003:
Total time elapsed: 0.048255205154418945 seconds

Bill HB1004:
Total time elapsed: 0.049263954162597656 seconds

Bill HB1005:
Total time elapsed: 0.05226325988769531 seconds

Bill HB1006:
Total time elapsed: 0.054346323013305664 seconds

Bill HB1007:
Total time elapsed: 0.05570840835571289 seconds

Bill HB1008:
Total time elapsed: 0.05747699737548828 seconds

Bill HB1009:
Total time elapsed: 0.06381869316101074 seconds

Bill HB101:
Total time elapsed: 0.08031868934631348 seconds

Bill HB1010:
Total time elapsed: 0.09134817123413086 seconds

Bill HB1011:
Total time elapsed: 0.09434723854064941 seconds

Bill HB1012:
Total time elapsed: 0.09725236892700195 seconds

Bill HB1

Total time elapsed: 0.8539960384368896 seconds

Bill HB1148:
Total time elapsed: 0.8539960384368896 seconds

Bill HB1149:
Total time elapsed: 0.8812859058380127 seconds

Bill HB115:
Total time elapsed: 0.8812859058380127 seconds

Bill HB1150:
Total time elapsed: 0.8812859058380127 seconds

Bill HB1151:
Total time elapsed: 0.9136734008789062 seconds

Bill HB1152:
Total time elapsed: 0.9136734008789062 seconds

Bill HB1153:
Total time elapsed: 0.9136734008789062 seconds

Bill HB1154:
Total time elapsed: 0.9450116157531738 seconds

Bill HB1155:
Total time elapsed: 0.9450116157531738 seconds

Bill HB1156:
Total time elapsed: 0.9757192134857178 seconds

Bill HB1157:
Total time elapsed: 0.9757192134857178 seconds

Bill HB1158:
Total time elapsed: 0.9757192134857178 seconds

Bill HB1159:
Total time elapsed: 1.0069518089294434 seconds

Bill HB116:
Total time elapsed: 1.0069518089294434 seconds

Bill HB1160:
Total time elapsed: 1.0069518089294434 seconds

Bill HB1161:
Total time elapsed: 1.0069

Total time elapsed: 1.4511828422546387 seconds

Bill HB1280:
Total time elapsed: 1.4533677101135254 seconds

Bill HB1281:
Total time elapsed: 1.4534926414489746 seconds

Bill HB1282:
Total time elapsed: 1.4940180778503418 seconds

Bill HB1283:
Total time elapsed: 1.4950299263000488 seconds

Bill HB1284:
Total time elapsed: 1.4970932006835938 seconds

Bill HB1285:
Total time elapsed: 1.4970932006835938 seconds

Bill HB1286:
Total time elapsed: 1.4970932006835938 seconds

Bill HB1287:
Total time elapsed: 1.4970932006835938 seconds

Bill HB1288:
Total time elapsed: 1.4970932006835938 seconds

Bill HB1289:
Total time elapsed: 1.4970932006835938 seconds

Bill HB129:
Total time elapsed: 1.4970932006835938 seconds

Bill HB1290:
Total time elapsed: 1.4970932006835938 seconds

Bill HB1291:
Total time elapsed: 1.4970932006835938 seconds

Bill HB1292:
Total time elapsed: 1.4970932006835938 seconds

Bill HB1293:
Total time elapsed: 1.5127208232879639 seconds

Bill HB1294:
Total time elapsed: 1.512

Total time elapsed: 2.079695701599121 seconds

Bill HB1437:
Total time elapsed: 2.0849006175994873 seconds

Bill HB1438:
Total time elapsed: 2.0849006175994873 seconds

Bill HB1439:
Total time elapsed: 2.0849006175994873 seconds

Bill HB144:
Total time elapsed: 2.0849006175994873 seconds

Bill HB1440:
Total time elapsed: 2.0849006175994873 seconds

Bill HB1441:
Total time elapsed: 2.0849006175994873 seconds

Bill HB1442:
Total time elapsed: 2.0849006175994873 seconds

Bill HB1443:
Total time elapsed: 2.096235752105713 seconds

Bill HB1444:
Total time elapsed: 2.0983219146728516 seconds

Bill HB1445:
Total time elapsed: 2.0993266105651855 seconds

Bill HB1446:
Total time elapsed: 2.1003379821777344 seconds

Bill HB1447:
Total time elapsed: 2.10532808303833 seconds

Bill HB1448:
Total time elapsed: 2.106316328048706 seconds

Bill HB1449:
Total time elapsed: 2.107332706451416 seconds

Bill HB145:
Total time elapsed: 2.1083192825317383 seconds

Bill HB1450:
Total time elapsed: 2.1093161106

Total time elapsed: 2.6557343006134033 seconds

Bill HB1606:
Total time elapsed: 2.688652276992798 seconds

Bill HB1607:
Total time elapsed: 2.7136013507843018 seconds

Bill HB1608:
Total time elapsed: 2.7136013507843018 seconds

Bill HB1609:
Total time elapsed: 2.7136013507843018 seconds

Bill HB161:
Total time elapsed: 2.7136013507843018 seconds

Bill HB1610:
Total time elapsed: 2.7292282581329346 seconds

Bill HB1611:
Total time elapsed: 2.7292282581329346 seconds

Bill HB1612:
Total time elapsed: 2.7448606491088867 seconds

Bill HB1613:
Total time elapsed: 2.7448606491088867 seconds

Bill HB1614:
Total time elapsed: 2.7448606491088867 seconds

Bill HB1615:
Total time elapsed: 2.776193380355835 seconds

Bill HB1616:
Total time elapsed: 2.776193380355835 seconds

Bill HB1617:
Total time elapsed: 2.776193380355835 seconds

Bill HB1618:
Total time elapsed: 2.776193380355835 seconds

Bill HB1619:
Total time elapsed: 2.7922959327697754 seconds

Bill HB162:
Total time elapsed: 2.792295932

...

Total time elapsed: 65.06341004371643 seconds

Bill SJR12:
Total time elapsed: 65.0832827091217 seconds

Bill SJR13:
Total time elapsed: 65.09042692184448 seconds

Bill SJR14:
Total time elapsed: 65.09042692184448 seconds

Bill SJR15:
Total time elapsed: 65.09042692184448 seconds

Bill SJR16:
Total time elapsed: 65.10678124427795 seconds

Bill SJR17:
Total time elapsed: 65.10678124427795 seconds

Bill SJR18:
Total time elapsed: 65.13753509521484 seconds

Bill SJR19:
Total time elapsed: 65.13753509521484 seconds

Bill SJR2:
Total time elapsed: 65.13753509521484 seconds

Bill SJR20:
Total time elapsed: 65.15315580368042 seconds

Bill SJR21:
Total time elapsed: 65.16887426376343 seconds

Bill SJR22:
Total time elapsed: 65.18450164794922 seconds

Bill SJR23:
Total time elapsed: 65.21619486808777 seconds

Bill SJR24:
Total time elapsed: 65.28175210952759 seconds

Bill SJR25:
Total time elapsed: 65.28973078727722 seconds

Bill SJR26:
Total time elapsed: 65.28973078727722 seconds

Bill SJR27:

Total time elapsed: 66.4945318698883 seconds

Bill SR179:
Total time elapsed: 66.49637508392334 seconds

Bill SR18:
Total time elapsed: 66.49740266799927 seconds

Bill SR180:
Total time elapsed: 66.49838757514954 seconds

Bill SR181:
Total time elapsed: 66.49938488006592 seconds

Bill SR182:
Total time elapsed: 66.50143218040466 seconds

Bill SR183:
Total time elapsed: 66.50259351730347 seconds

Bill SR184:
Total time elapsed: 66.50259351730347 seconds

Bill SR185:
Total time elapsed: 66.50259351730347 seconds

Bill SR186:
Total time elapsed: 66.52018451690674 seconds

Bill SR187:
Total time elapsed: 66.52120542526245 seconds

Bill SR188:
Total time elapsed: 66.5367534160614 seconds

Bill SR189:
Total time elapsed: 66.53784346580505 seconds

Bill SR19:
Total time elapsed: 66.5447769165039 seconds

Bill SR190:
Total time elapsed: 66.54678916931152 seconds

Bill SR191:
Total time elapsed: 66.54986143112183 seconds

Bill SR192:
Total time elapsed: 66.552001953125 seconds

Bill SR193:
Tota

Total time elapsed: 67.26705932617188 seconds

Bill SR352:
Total time elapsed: 67.27105116844177 seconds

Bill SR353:
Total time elapsed: 67.27931714057922 seconds

Bill SR354:
Total time elapsed: 67.28035163879395 seconds

Bill SR355:
Total time elapsed: 67.28559303283691 seconds

Bill SR356:
Total time elapsed: 67.28659868240356 seconds

Bill SR357:
Total time elapsed: 67.28859877586365 seconds

Bill SR358:
Total time elapsed: 67.28960180282593 seconds

Bill SR359:
Total time elapsed: 67.29260182380676 seconds

Bill SR36:
Total time elapsed: 67.29495239257812 seconds

Bill SR360:
Total time elapsed: 67.29694724082947 seconds

Bill SR361:
Total time elapsed: 67.29994654655457 seconds

Bill SR362:
Total time elapsed: 67.30098533630371 seconds

Bill SR363:
Total time elapsed: 67.3023407459259 seconds

Bill SR364:
Total time elapsed: 67.3023407459259 seconds

Bill SR365:
Total time elapsed: 67.3035535812378 seconds

Bill SR366:
Total time elapsed: 67.3035535812378 seconds

Bill SR367:
To

Total time elapsed: 67.9653971195221 seconds

Bill SR514:
Total time elapsed: 67.96844577789307 seconds

Bill SR515:
Total time elapsed: 67.97146773338318 seconds

Bill SR516:
Total time elapsed: 67.97146773338318 seconds

Bill SR517:
Total time elapsed: 67.97146773338318 seconds

Bill SR518:
Total time elapsed: 67.97146773338318 seconds

Bill SR519:
Total time elapsed: 67.97146773338318 seconds

Bill SR52:
Total time elapsed: 67.97146773338318 seconds

Bill SR520:
Total time elapsed: 67.97146773338318 seconds

Bill SR521:
Total time elapsed: 67.98845863342285 seconds

Bill SR522:
Total time elapsed: 67.98936676979065 seconds

Bill SR523:
Total time elapsed: 68.02193093299866 seconds

Bill SR524:
Total time elapsed: 68.02493333816528 seconds

Bill SR525:
Total time elapsed: 68.02693057060242 seconds

Bill SR526:
Total time elapsed: 68.02893137931824 seconds

Bill SR527:
Total time elapsed: 68.03093671798706 seconds

Bill SR528:
Total time elapsed: 68.03393983840942 seconds

Bill SR529:

Total time elapsed: 68.73381304740906 seconds

Bill SR673:
Total time elapsed: 68.76322841644287 seconds

Bill SR674:
Total time elapsed: 68.7730860710144 seconds

Bill SR675:
Total time elapsed: 68.77708077430725 seconds

Bill SR676:
Total time elapsed: 68.78125500679016 seconds

Bill SR677:
Total time elapsed: 68.7969810962677 seconds

Bill SR678:
Total time elapsed: 68.7969810962677 seconds

Bill SR679:
Total time elapsed: 68.7969810962677 seconds

Bill SR68:
Total time elapsed: 68.7969810962677 seconds

Bill SR680:
Total time elapsed: 68.7969810962677 seconds

Bill SR681:
Total time elapsed: 68.81258630752563 seconds

Bill SR682:
Total time elapsed: 68.81258630752563 seconds

Bill SR683:
Total time elapsed: 68.81258630752563 seconds

Bill SR684:
Total time elapsed: 68.81258630752563 seconds

Bill SR685:
Total time elapsed: 68.82820630073547 seconds

Bill SR686:
Total time elapsed: 68.82820630073547 seconds

Bill SR687:
Total time elapsed: 68.82820630073547 seconds

Bill SR688:
Tota

In [7]:
bills[3257]["became_law"]["signed"] = True
bills[3257]["became_law"]["date"] = "2024-05-16"
bills[14023]["became_law"]["signed"] = True
bills[14023]["became_law"]["date"] = "2024-05-24"

Check that we got all the bills that became law (there should be 64 as of 1 June 2024).

In [8]:
print("\nNumber of bills that became law:")
# Make bills into a data frame
bills_df = pd.json_normalize(bills)
print(bills_df["became_law.signed"].value_counts()[1])


Number of bills that became law:
64


Finally, we create a file with the dictionaries.

In [9]:
# Time the execution
start = time.time() # for debugging
print("Saving the file now...") # for debugging

# Change to a json string    
bills_string = json.dumps(bills)

# Save parsed string to a .json file
save_file = open("initial_data_cleaning.json", "w")  
json.dump(json.loads(bills_string), save_file, indent = 4)  
save_file.close()

# Time elapsed
end = time.time() # for debugging
print("Time to create/overwrite the file: "+str(end-start)+" seconds.") # for debugging
print("Total time: "+str(total_time+end-start)+" seconds.") # for debugging

Saving the file now...
Time to create/overwrite the file: 15.129270076751709 seconds.
Total time: 84.18670463562012 seconds.
